In [3]:
import xarray as xr
import rioxarray
import rasterio
from rasterio.transform import from_origin

# ==== User settings ====
input_nc = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack.tif"
variables_to_process = ['t2m', 'd2m', 'u10', 'v10', 'tp']  # ERA5 variables
target_resolution = 30  # meters

# Open dataset
ds = xr.open_dataset(input_nc)

# Choose an appropriate projected CRS (replace with your area's UTM zone if known)
# Example: EPSG:32644 is UTM zone 44N; you must change it to match your region
target_crs = "EPSG:32644"

processed_bands = []

for var in variables_to_process:
    print(f"Processing variable: {var}")
    
    # Extract variable as DataArray
    da = ds[var]
    
    # Select first time step for demonstration (adjust if using time series)
    da = da.isel(valid_time=0)  
    
    # Attach spatial dimensions to rioxarray
    da = da.rio.write_crs("EPSG:4326")  # Original is lat/lon WGS84
    
    # Reproject to target CRS (UTM meters) and resample to 30 m
    da_utm = da.rio.reproject(
        target_crs,
        resolution=target_resolution,
        resampling=rasterio.enums.Resampling.bilinear
    )
    
    processed_bands.append(da_utm)

# Stack all processed bands into one DataArray
stacked = xr.concat(processed_bands, dim="band")
stacked = stacked.assign_coords(band=range(1, len(variables_to_process) + 1))

# Save to one multi-band GeoTIFF
stacked.rio.to_raster(output_tif)

print(f"✅ Multi-band raster saved at: {output_tif}")

Processing variable: t2m
Processing variable: d2m
Processing variable: u10
Processing variable: v10
Processing variable: tp
✅ Multi-band raster saved at: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack.tif
